In [ ]:
!mkdir cifar && cd cifar && wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xvf cifar/cifar-10-python.tar.gz -C cifar/ --no-same-owner
!mkdir cifar/data

import os
import platform
import numpy as np
from PIL import Image
from six.moves import cPickle as pickle

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def load_pickle(f):
    return  pickle.load(f, encoding='latin1')

def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        N = datadict['filenames']
        X = X.reshape(10000,3072)
        Y = np.array(Y)
        return X, Y, N


path = 'cifar'
location = os.path.join(path, 'cifar-10-batches-py/test_batch')
x, y, n = load_CIFAR_batch(location)

for idx, file in enumerate(x):
    img = Image.fromarray(file.reshape((3,32,32)).transpose((1,2,0)))
    if not os.path.exists(os.path.join(path, 'data', str(y[idx]))):
        os.makedirs(os.path.join(path, 'data', str(y[idx])))
    img.save(os.path.join(path, 'data', str(y[idx]), str(n[idx])[:-4] + '.bmp'), format='BMP') 
print(f"Completed! {[len([f for f in os.listdir(os.path.join('/content/cifar/data/', str(i)))]) for i in range(10)]}")


[os.makedirs(os.path.join('/content/cifar/test_data/', folder)) for folder in ['planes', 'cars', 'val/planes', 'val/cars', 'train/planes', 'train/cars']]
!cp -r cifar/data/0/* cifar/test_data/planes && cp -r cifar/data/1/* cifar/test_data/cars
!ls -rt /content/cifar/test_data/planes/* | head -200 | xargs mv -t /content/cifar/test_data/val/planes
!ls -rt /content/cifar/test_data/planes/* | head -800 | xargs mv -t /content/cifar/test_data/train/planes
!ls -rt /content/cifar/test_data/cars/* | head -200 | xargs mv -t /content/cifar/test_data/val/cars
!ls -rt /content/cifar/test_data/cars/* | head -800 | xargs mv -t /content/cifar/test_data/train/cars
!rm -rf /content/cifar/test_data/cars/ && rm -rf /content/cifar/test_data/planes/
print(f"Completed! {[len([f for f in os.listdir(os.path.join('/content/cifar/test_data', str(i)))]) for i in ['val/planes', 'val/cars', 'train/planes', 'train/cars']]}")

# Новый раздел

1.b Загрузить один из известных наборов данных (Cifar, Mnist, ImageNet...) из torchvision. Далее работать с ним при помощи инструментов pyTorch:

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import os

path = './1_b_torchvision'
if not os.path.exists(path):
    os.makedirs(path)

trans = transforms.Compose(([
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    ]))
data = datasets.CIFAR100(path, train=True, transform=trans, download=True)
dloader = DataLoader(data, batch_size=4, shuffle=True, num_workers=2) 
class_names = data.classes
print(data.classes)


In [ ]:
import torchvision 
import numpy as np
import matplotlib.pyplot as plt

def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    #inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dloader))
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

1.a Скачать один из известных наборов данных (Cifar, Mnist, ImageNet...) в оригинальном виде:

In [ ]:
def download(url, filename, chunk_size=1024):
    with open(filename, "wb") as fh:
        with urllib.request.urlopen(urllib.request.Request(url)) as response: 
            with tqdm(total=response.length) as pbar:
                for chunk in iter(lambda: response.read(chunk_size), ""):
                    if not chunk:
                        break
                    pbar.update(chunk_size)
                    fh.write(chunk)

In [ ]:
import urllib
from tqdm import tqdm
import pickle
import os

path = './1_a_onw_tools'
if not os.path.exists(path):
    os.makedirs(path)

download('https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz', os.path.join(path, 'cifar_10.tar.gz'))

In [ ]:
import tarfile
my_tar = tarfile.open(os.path.join(path, 'cifar_10.tar.gz'))
my_tar.extractall(path) 
my_tar.close()

In [ ]:
def get_data(path_to_file: str):
    data: Any = []
    targets = []

    file_path = os.path.join(path_to_file)
    with open(file_path, 'rb') as f:
        entry = pickle.load(f, encoding='latin1')
        data.append(entry['data'])
        if 'labels' in entry:
            targets.extend(entry['labels'])
        else:
            targets.extend(entry['fine_labels'])

    data = np.vstack(data).reshape(-1, 3, 32, 32) 
    data = data.transpose((0, 2, 3, 1))  # convert to HWC
    return data, np.array(targets)

In [ ]:
img, lbl = get_data(os.path.join(path, 'cifar-10-batches-py/test_batch'))
print(f'DATA: {img.shape}, {lbl.shape}')
class_names = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Hаписать собственный класс Dataset, так чтобы выход был совместим с torch.utils.data.Dataloader

In [ ]:
class CustomDataset():
    def __init__(self, images, labels):
        self.labels = labels
        self.images = self.prepare_image(images)

    def prepare_image(self, img):
        img = img.transpose((0,3,1,2))
        img = img/255.
        return img

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        label = self.labels[idx]
        images = self.images[idx]
        return images, label

In [ ]:
from torch.utils.data import DataLoader
dataset = CustomDataset(img, lbl)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2) 

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloader))
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

2.а Попробовать предварительно обработать данные, так, чтобы их можно было загрузить при помощи torchvision.Datasets (torchvision.datasets.DatasetFolder)

In [ ]:
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(32),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(32),
        #transforms.CenterCrop(32),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'cifar/test_data/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

Преобразования проводимые с изображениями

In [ ]:
import numpy as np
from skimage.transform import rescale, rotate
import random
import numbers
import os
from PIL import Image, ImageOps
import collections

class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, data):
        for t in self.transforms:
           data = t(data)
        return data


class Scale(object):

    def __init__(self, scale):
        self.scale = scale

    def __call__(self, sample):
        image, label = sample
        image = np.asarray(image) #Image.fromarray
        img_size = image.shape[0]

        scale = np.random.uniform(low=1.0 - self.scale, high=1.0 + self.scale)
        
        image = rescale(
            image,
            (scale, scale),
            multichannel=True,
            preserve_range=True,
            mode="constant",
            anti_aliasing=False,
        )

        if scale < 1.0:
            diff = (img_size - image.shape[0]) / 2.0
            padding = ((int(np.floor(diff)), int(np.ceil(diff))),) * 2 + ((0, 0),)
            image = np.pad(image, padding, mode="constant", constant_values=0)
        else:
            x_min = (image.shape[0] - img_size) // 2
            x_max = x_min + img_size
            image = image[x_min:x_max, x_min:x_max, ...]

        image = Image.fromarray(image.astype('uint8'))
        return image, label


class Rotate(object):
    def __init__(self, angle):
        self.angle = angle
        self.topil = toPIL()

    def __call__(self, sample):
        image, label = sample
        angle = np.random.uniform(low=-self.angle, high=self.angle)
        image, label = self.topil(sample)
        image = image.rotate(angle=angle)    
        return image, label


class Normalize(object):
    def __init__(self, std, mean):
        self.std = std
        self.mean = mean

    def __call__(self, sample):
        image, label = sample
        image = np.array(image, dtype=np.float32)
        image = image / 255
        image[:,:,0] = (image[:,:,0] - self.mean[0]) / self.std[0]
        image[:,:,1] = (image[:,:,1] - self.mean[1]) / self.std[1]
        image[:,:,2] = (image[:,:,2] - self.mean[2]) / self.std[2]
        image = Image.fromarray((image*255).astype('uint8'))
        return image, label


class HorizontalFlip(object):
    def __init__(self, flip_prob):
        self.flip_prob = flip_prob

    def __call__(self, sample):
        image, label = sample
        if np.random.rand() < self.flip_prob:
            return image, label
        img = image.transpose(method=Image.FLIP_TOP_BOTTOM)
        return img, label


class RandomCrop(object):
    def __init__(self, size, padding=0):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size
        self.padding = padding

    @staticmethod
    def get_params(img, output_size):
        w, h = img.size
        th, tw = output_size
        if w == tw and h == th:
            return img
        if w > tw and h > th:
            i = random.randint(0, th)
            j = random.randint(0, tw)
        else:
            i = random.randint(0, h - th)
            j = random.randint(0, w - tw)
        return i, j, th, tw

    def pad(self, img, padding, fill=0):
        if not isinstance(img, Image.Image):
            raise TypeError('img should be PIL Image. Got {}'.format(type(img)))

        if not isinstance(padding, (numbers.Number, tuple)):
            raise TypeError('Got inappropriate padding arg')
        if not isinstance(fill, (numbers.Number, str, tuple)):
            raise TypeError('Got inappropriate fill arg')

        if isinstance(padding, collections.Sequence) and len(padding) not in [2, 4]:
            raise ValueError("Padding must be an int or a 2, or 4 element tuple, not a " +
                            "{} element tuple".format(len(padding)))

        return ImageOps.expand(img, border=padding, fill=fill)

    def __call__(self, sample):
        image, label = sample
        if self.padding > 0:
            image = self.pad(image, self.padding)
        i, j, h, w = self.get_params(image, self.size)
        img = image.crop((i, j, i+h, j+w))
        return img, label


class toNumpy():
    def __init__(self):
        pass
    def __call__(self, sample):
        image, label = sample
        if not isinstance(image, (np.ndarray, np.generic) ):
            image = np.asarray(image)
        image = np.transpose(image, (2,0,1))
        return image, label

    
class toPIL():
    def __init__(self):
        pass
    def __call__(self, sample):
        image, label = sample        
        if isinstance(image, (Image.Image)):
            return image, label
        elif isinstance(image, (np.ndarray, np.generic)):
            if image.shape[2] != 3:
                image = image.transpose((1,2,0))
            image = Image.fromarray(image.astype('uint8'))
            return image, label
        else:
            raise TypeError(f'Unsupported type {type(image)} for this operation')

In [ ]:
!mkdir image && cd image && wget https://lh3.googleusercontent.com/sZl-o9TGNYXucrPNHesxaMLXajhcPYqw43zojzHJ-y1yYYeQaNtJMrPUQImKgak3paKOMbEm0Av0e5bKG8_z31m1xVGN8J3x-EIAfgeETIhrLxwsw7xWEGstIuwyKYuHjOLFPCcvqIBY944PWFHBhgSEsVERXJljVEwPTD1xpJHhi5gHieiZcjl-rJ734bFiDxG1GzTxkX5nRc9lkRhtNHMdHDOSswMU-dgp8itMF8lTdEztOYz_bE_8H2FxN5NtCBmeOvxTi7f31wM2zrAE7oOzzeVy1_hYLFAWuXJ0CNqDfc-J-Ui9HY1RILj9Z1nYvEKGGDSTiT3tzysWHO9Vn6rXDFKE9TUGNE1_Z9_EaQ7B_HIU_z7oq2Hmmikl1Ap5t7N_pEI44ZhUGbIVirHKJyvc2LgtJgczCeNEgH7SnvvYBCM-OMWX5MnB949rXYn678iZyz7Q16wjPMevr1IQU4zfdOE2xoJQi8x3FhpD2-3moUoilZRBR5OQi-KII4hRYrEaRFnAiGqYeFz3Rqwx_Yw97kROwh2AnfhT03700AHkFqmYobTXjq0Q3IQpjcYQZ3vbtw=w800-h400-p && mv ./* image.jpg


In [ ]:
import random
img = Image.open('./image/image.jpg')
label = np.array([1])
# transform = Compose([RandomCrop(200, padding=50)])
# image, label = transform((img, label))
# transform = Compose([HorizontalFlip(0.5)])
# image, label = transform((img, label))
# transform = Compose([Rotate(90)])
# image, label = transform((img, label))
# transform = Compose([Scale(0.9)])
# image, label = transform((img, label))
# transform = Compose([Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
# image, label = transform((img, label))                    
# transform = Compose([toNumpy()])                    
# image, label = transform((img, label))

# transform = Compose([RandomCrop(200, padding=0),
#                     HorizontalFlip(0.5),
#                     Rotate(90),
#                     Scale(0.9),
#                     Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
#                     ])
# image, label = transform((img, label))

display(image)

Загрузчики данных

In [ ]:
def get_data(path_to_file: str):
    data: Any = []
    targets = []

    file_path = os.path.join(path_to_file)
    with open(file_path, 'rb') as f:
        entry = pickle.load(f, encoding='latin1')
        data.append(entry['data'])
        if 'labels' in entry:
            targets.extend(entry['labels'])
        else:
            targets.extend(entry['fine_labels'])

    data = np.vstack(data).reshape(-1, 3, 32, 32) 
    data = data.transpose((0, 2, 3, 1))  # convert to HWC
    return data, np.array(targets)

In [ ]:
import pickle
import numpy as np

path = './cifar'
imgs, lbls = get_data(os.path.join(path, 'cifar-10-batches-py/test_batch'))
print(f'DATA: {imgs.shape}, {lbls.shape}')
class_names = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
class CustomDataset():
    def __init__(self, images, labels, transforms=None):
        self.labels = labels
        self.images = images #self.prepare_image(images)
        self.transforms = transforms

    def prepare_image(self, img):
        img = img.transpose((2,0,1))
        img = img/255.
        return img

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        label = self.labels[idx]
        image = self.images[idx]
        if self.transforms is not None:
            image, label = self.transforms([image, label])
            image = image / 255
        else:
            image = self.prepare_image(image)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.int32).unsqueeze(-1)

In [ ]:
def own_collate(batch):
    if isinstance(batch, list):
        for el in batch:
            if isinstance(el, tuple) and len(el) == 2:
                pass
            else:
                raise TypeError(f'Unsupported element {type(el)} for length {len(el)}')
        elements = list(map(lambda x: list(x), zip(*batch)))
    return elements

In [ ]:
from torch.utils.data import DataLoader
import torchvision
import torch

transform = Compose([Rotate(90),
                     toNumpy()])

dataset = CustomDataset(imgs, lbls, transforms=transform)

dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0, sampler=None,
                        batch_sampler=None, collate_fn=own_collate, pin_memory=False, drop_last=False, timeout=0,
                        worker_init_fn=None, prefetch_factor=2, persistent_workers=False) 

# Get a batch of training data
inputs, classes = next(iter(dataloader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
imshow(out, title=[class_names[x] for x in classes])

Tensorboard

In [ ]:
%load_ext tensorboard

import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets

# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

# model = torch.nn.Sequential(torch.nn.Conv2d(3, 64, 3),
#                           torch.nn.BatchNorm2d(64),
#                           torch.nn.ReLU(),
#                           torch.nn.Conv2d(64, 32, 3),
#                           torch.nn.BatchNorm2d(32),
#                           torch.nn.ReLU(),
#                           torch.nn.Conv2d(32, 16, 3),
#                           torch.nn.BatchNorm2d(16),
#                           torch.nn.ReLU()
#                           )
model = torchvision.models.resnet50()

image = torch.stack(inputs)
grid = torchvision.utils.make_grid(image)
writer.add_image('images', grid, 0)
writer.add_graph(model, image)
writer.close()

In [ ]:
%tensorboard --logdir=runs

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np

writer = SummaryWriter()

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

In [ ]:
%tensorboard --logdir=runs

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
r = 5
for i in range(100):
    writer.add_scalars('run_14h', {'xsinx':i*np.sin(i/r),
                                    'xcosx':i*np.cos(i/r),
                                    'tanx': np.tan(i/r)}, i)
writer.close()

In [ ]:
%tensorboard --logdir=runs

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch
import os
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

writer = SummaryWriter()

path = './1_b_torchvision'
if not os.path.exists(path):
    os.makedirs(path)

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
    ])

data = datasets.CIFAR100(path, train=True, transform=transform, download=True)
dloader = DataLoader(data, batch_size=4, shuffle=True, num_workers=2) 
class_names = data.classes
print(data.classes)


def select_n_random(data, labels, n=100):
    '''
    Selects n random datapoints and their corresponding labels from a dataset
    '''
    assert len(data) == len(labels)
    perm = torch.randperm(len(data))
    return torch.tensor(data)[perm][:n], torch.tensor(labels)[perm][:n]

# select random images and their target indices
images, labels = select_n_random(data.data, data.targets)

images = images.transpose(1,3).transpose(2,3)/255.
# get the class labels for each image
class_labels = [class_names[lab] for lab in labels]

# log embeddings
print(f'Is data in image contiguous: {images.is_contiguous()}')
features = images.view(-1, 32 * 32 * 3) #.contiguous().
writer.add_embedding(features,
                    metadata=class_labels,
                    label_img=images) 
writer.close()

In [ ]:
%tensorboard --logdir=runs

In [ ]:
!rm -rf ./runs/*

In [ ]:
import math
import torch
from torch.utils.tensorboard import SummaryWriter

class Model:
  def __init__(self):
      self.params = [torch.randn((), device=device, dtype=dtype, requires_grad=True) for i in range(params_num)]

  def get_params(self):
      return self.params

  def __call__(self, x):
      return reduce(lambda x,y: x+y, [p*x**mod for mod, p in enumerate(self.params)])  

writer = SummaryWriter()

dtype = torch.float
device = torch.device('cpu')

x = torch.linspace(-math.pi, math.pi, 3000, device=device, dtype=dtype)
y = torch.sin(x)
params_num = 4
model = Model()

from functools import reduce
learning_rate = 1e-4
running_loss = 0.0
criterion = torch.nn.MSELoss()
optim = torch.optim.Adam(model.get_params(), lr=learning_rate)
for t in range(10000):
    optim.zero_grad()
    y_pred = model(x) 
    loss = criterion(y_pred, y)
    loss.backward()   
    optim.step()

    running_loss += loss.item()
    if t % 100 == 99:
        writer.add_scalar('training loss',
                                running_loss/100,
                                2000 + t)
        writer.add_scalar('loss',
                                loss.item(),
                                2000 + t)
        running_loss = 0.0

print(f'Training is completed')


In [ ]:
%tensorboard --logdir=runs